In [ ]:
from modeling.gen6.api import symbolic, log
from modeling.gen7.api7 import MFunctionalSet, MFunctionalSetLeaf
from engine.torchdata import print_formatted_table, load_vals
import numpy as np

In [6]:
m_l1, m_l2, m_l3, m_s1, m_s2, v_exh, alpha = 100, 100, 100, 10, 10, 4000, 0.6
m_s3, m_i1, m_f1, m_i2, m_f2, m_i3, m_f3, dv1, dv2, dv3, dvtot, m_t = symbolic(
    'm_s3', 'm_i1', 'm_f1', 'm_i2', 'm_f2', 'm_i3', 'm_f3', '\\Delta v_1',
    '\\Delta v_2', '\\Delta v_3', '\\Delta v', 'm_t'
)
D1 = MFunctionalSetLeaf(m_s3 == alpha*m_t, idvals=[0]) 
D2 = MFunctionalSetLeaf(m_t == m_l1+m_s1+m_l2+m_s2+m_l3+m_s3, idvals=[1])
D3 = MFunctionalSetLeaf(m_i1 == m_t, idvals=[2])
D4 = MFunctionalSetLeaf(m_f1 == m_i1-m_l1, idvals=[3])
D5 = MFunctionalSetLeaf(m_i2 == m_f1-m_s1, idvals=[4])
D6 = MFunctionalSetLeaf(m_f2 == m_i2-m_l2, idvals=[5])
D7 = MFunctionalSetLeaf(m_i3 == m_f2-m_s2, idvals=[6])
D8 = MFunctionalSetLeaf(m_f3 == m_i3-m_l3, idvals=[7])
D9 = MFunctionalSetLeaf(dv1 == v_exh*log(m_i1/m_f1), idvals=[8])
D10 = MFunctionalSetLeaf(dv2 == v_exh*log(m_i2/m_f2), idvals=[9])
D11 = MFunctionalSetLeaf(dv3 == v_exh*log(m_i3/m_f3), idvals=[10])
D12 = MFunctionalSetLeaf(dvtot == dv1+dv2+dv3, idvals=[11])

Loop = MFunctionalSet(D1,D2)
FPF = MFunctionalSet(D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12)

In [7]:
f1 = Loop.config(residuals=[D1,D2]).build()
x0 = load_vals({'m_t': 10, 'm_s3': 10}, f1.indices, isdict=True)
xsol = f1.analysis(x0)
idxrev = {var.item():key for key,var in f1.indices.items()}
print_formatted_table([np.round(xsol,2)], f1.indices, idxrev)

m_t m_s3
800 480 


In [8]:
f2 = FPF.config(elim=[Loop.config(residuals=[D1,D2]), D3, D4, D5, D6, D7, D8, D9, D10, D11, D12]).build()
x0 = load_vals({'m_t': 10, 'm_s3': 10}, f2.indices, isdict=True)
xsol = f2.analysis(x0)
idxrev = {var.item():key for key,var in f2.indices.items()}
print_formatted_table([np.round(xsol,2)], f2.indices, idxrev)

m_f2 m_i2 \Delta v_3 m_i3 m_s3 m_f3 m_i1 m_f1 m_t \Delta v_2 \Delta v \Delta v_1
590  690  756.97     580  480  480  800  700  800 626.28     1917.37  534.13    
